In [27]:
import os
import glob
import sys
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from agents import Agent, Runner, function_tool
from agents.tracing import trace
from datetime import datetime


In [28]:
MODEL = "gpt-5.2-2025-12-11"
db_name = "vector_db"
load_dotenv(override=True)
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")



OpenAI API Key exists and begins sk-proj-


In [29]:

folders = glob.glob("knowledge-base/*")

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)


In [30]:
print(f"Loaded {len(documents)} documents")

header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("###", "section")
    ]
)
chunks = []

for doc in documents:
    split_docs = header_splitter.split_text(doc.page_content)
    chunks.extend(split_docs)


print(f"Divided into {len(chunks)} chunks\n")


for i, chunk in enumerate(chunks):
        print(f"--- Chunk {i+1} ---")
        print(chunk.page_content)
        print("\n")

# need to fix chunking

Loaded 7 documents
Divided into 61 chunks

--- Chunk 1 ---
Through escrow accounts:
- All buyer payments go into escrow
- Funds released based on construction progress  
Source: https://dlp.dubai.gov.ae/


--- Chunk 2 ---
A regulated account where:
- Your money is held securely
- Developer cannot freely access funds


--- Chunk 3 ---
- Construction
- Consultants
- Marketing (max 5%)


--- Chunk 4 ---
| Completion Level | Developer Right |
|------------------|----------------|
| ≥80% complete | Keep all payments |
| ≥60% complete | Keep up to 40% |  
Source: https://dlp.dubai.gov.ae/


--- Chunk 5 ---
- No escrow withdrawals until 20% completion
- Escrow funds protected from seizure
- Buyers receive financial reporting  
Source: https://adrec.gov.ae/


--- Chunk 6 ---
In Abu Dhabi:
- If delay > 6 months
- Authorities may impose penalties in favor of buyers


--- Chunk 7 ---
Possible outcomes:
- New developer completes project
- Refund via escrow
- Third-party intervention


--- Chunk 8 

In [31]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()
     # build vector db for KB

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")
retriever = vectorstore.as_retriever()


Vectorstore created with 61 documents


In [32]:
questionn = "UAE golden visa requirements and process for real estate investors, especially off-plan property purchase: eligibility thresholds, property value, mortgage rules, joint ownership, emirate differences, application steps and fees"
docs1 = retriever.invoke(questionn)
print(docs1)

[Document(id='47d69f38-f478-4ebb-ae06-da854cab9bba', metadata={}, page_content='# UAE Golden Visa — Real Estate Investor Route  \n## Overview\nIf you own a property (or group of properties) in the UAE, you may be eligible for a Golden Visa. Mortgaged properties may be accepted.  \n## Eligibility'), Document(id='501f7d34-4a3c-4c09-bb86-423b786967ce', metadata={}, page_content='# Visa Requirements for Off-Plan Property Investors (UAE)'), Document(id='3f89bf62-c6da-43f5-979c-fdb33d658246', metadata={'section': 'What is the Golden Visa requirement for property investors?'}, page_content='- Minimum property value: AED 2 million\n- Can include multiple properties\n- Must be in your name\n- Mortgages allowed (with bank NOC)  \nSource: https://dubailand.gov.ae/en/eservices/request-for-golden-visa-investor/'), Document(id='6bd94a87-8221-4cc7-aeec-372d297c6f8e', metadata={}, page_content='# UAE Real Estate Residence Visa  \n## Overview\nA foreigner who owns real estate in the UAE can obtain a re

In [ ]:
@function_tool
def search_docs(question: str) -> str:
    """
    Use this tool to answer any questions related to laws, regulations, visas, taxes, fees, financing, or ownership rules for UAE real estate. 
    Always rely on this tool for factual or legal information. Do not generate answers from memory. 
    If the tool returns low confidence or no results, ask the user for clarification instead of guessing.
    RAG database can only be queried in english 
    """
    docs = retriever.invoke(question)
    return "\n\n".join(doc.page_content for doc in docs)


prompt = """
You are a knowledgeable, friendly assistant representing real estate broker who speciallises in off-plan properties in the UAE.
You have a nuanced understanding of the nuances of the UAE market and the rules for the golden visa.
You are chatting with foreign investors who are interested in buying off-plan properties in the UAE.
Use the search_docs tool to retrieve to answer any questions related to laws, regulations, visas, taxes, fees, financing, or ownership rules for UAE real estate from the curated knowledge base.Answers should be clear and concise to not ramble. Answer the users question directly. 
"""

In [34]:

tools = [search_docs]

agent = Agent(
    name="Broker Agent",
    instructions=prompt,
    tools=tools,
    model=MODEL
)

trace_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

with trace(trace_id):
    result = await Runner.run(agent, "how to get golden visa")
    
result.final_output

'To get a UAE **Golden Visa via real estate**, the main route is owning UAE property with a qualifying value.\n\n### Eligibility (property investor)\n- **Minimum property value:** **AED 2,000,000**\n- **Can be one or multiple properties** (combined value can count)\n- **Property must be in your name**\n- **Mortgages are allowed** in many cases — typically you’ll need a **bank NOC (No Objection Certificate)** confirming the mortgage/paid amount as required\n\n(Source: Dubai Land Department “Request for Golden Visa Investor” e-service)\n\n### Typical process (high level)\n1. **Own qualifying property** (ready or off-plan, as long as it meets the requirements and is in your name)\n2. Apply through the relevant government channel (in Dubai, commonly via **Dubai Land Department** service)\n3. After initial approval, you complete the **residency issuance steps** (medical, Emirates ID, etc.). You’re generally allowed to stay **up to 60 days from entry** while completing issuance procedures.\n